In [6]:
from liquidagent.config import (
    LiquidAgentConfig,
    OllamaProviderConfig,
    SupportedModelProviders,
)
from liquidagent.tools.simple_chembl_search import search_similar_compounds_sync
from liquidagent.tools.surechembl_search import get_patents_from_smiles
from liquidagent.utils import get_chat_model, process_out

Testing out the liquidagent api

In [7]:
config = LiquidAgentConfig(
    preferred_model=SupportedModelProviders("ollama"),
    ollama=OllamaProviderConfig(
        host="http://karkinos:11434",
        model="devstral",
    ),
)

In [8]:
# Tools can be manually defined and passed into chat

"""
Search ChEMBL for compounds similar to the input SMILES.

Args:
    smiles: SMILES string of the compound to search for
    threshold: Similarity threshold (0-100, default 100 for exact match)
    max_results: Max number of results to return (default 20)
    timeout: Request timeout in seconds

Returns:
    Dictionary containing:
    - results: List of compounds with ChEMBL IDs, SMILES, similarity scores
    - total_found: Total number of compounds found
    - error: Error message if search failed
"""
search_similar_compounds_tools = {
    "type": "function",
    "function": {
        "name": "search_similar_chemical_compounds",
        "description": "Search ChEMBL for compounds similar to the input SMILES",
        "parameters": {
            "type": "object",
            "required": ["smiles"],
            "properties": {
                "smiles": {
                    "type": "string",
                    "description": "SMILES string of the compound to search for",
                },
            },
        },
    },
}
get_patents_from_smiles_tool = {
    "type": "function",
    "function": {
        "name": "get_patents_from_smiles",
        "description": "Search SureChEMBL for patents using the input SMILES",
        "parameters": {
            "type": "object",
            "required": ["smiles"],
            "properties": {
                "smiles": {
                    "type": "string",
                    "description": "SMILES string of the compound to retrieve patents for",
                },
            },
        },
    },
}

available_functions = {
    "get_patents_from_smiles": get_patents_from_smiles,
}

In [9]:
llm = get_chat_model(config)
llm.bind_tools([get_patents_from_smiles])
system_prompt = """
You are a specialized assistant who can help users retrieve patent information for a given chemical compound using the tools available to you.
After retrieving patents, only summarize chemically relevant patents for the user
"""
messages = [
    {
        "content": system_prompt,
        "role": "system",
    },
    {
        "content": "Search patents for the chemical compound CC(=O)OC1=CC=CC=C1C(=O)O",
        "role": "user",
    },
]

In [10]:
resp_messages = llm.agent(messages, True, available_functions)

2025-06-23 20:12:07,625 - liquidagent.chat_model.ollama - INFO - Messages len: 2
2025-06-23 20:12:08,399 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Calling function: get_patents_from_smiles
Arguments: {'smiles': 'CC(=O)OC1=CC=CC=C1C(=O)O'}


2025-06-23 20:12:39,352 - liquidagent.chat_model.ollama - INFO - Messages len: 4
2025-06-23 20:12:42,703 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


It appears that you have a complex JSON structure containing various sections related to patents. The data includes abstracts, descriptions, claims, and annotations of specific chemicals mentioned within the text.

To analyze or interpret this data further, here are some key points broken down:

### Key Sections

1. **Abstract**:
   - Describes the invention generally.
   - Mentions a western medicine frame that is multi-functional and movable.

2. **Descriptions**:
   - Detailed explanations of how each component works within the patented device.
   - Includes diagrams or illustrations (though not shown in the text).
   - Specific components like anti-collision plates, placing cabinets, western medicine racks, and air suction pumps are detailed.

3. **Claims**:
   - Legal definitions that outline what is claimed as new by the inventor.
   - Claim 1 describes a multi-functional movable western medicine frame with several specific parts (base, anti-collision plates, placing cabinet, etc